In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


df = pd.read_csv('C:/Users/vdzit/Desktop/adult.csv', sep=",")

In [3]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'education']
le = LabelEncoder()

for col in categorical_columns:
    df[col] = le.fit_transform(df[col])


In [4]:
X = df.drop('income', axis=1)  
y = df['income']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
knn_base = KNeighborsClassifier()

# Определим набор гиперпараметров для GridSearchCV
param_grid = {'n_neighbors': range(1, 31)}  # Попробуем значения K от 1 до 30

# Создадим объект GridSearchCV
grid_search = GridSearchCV(knn_base, param_grid, cv=5, scoring='accuracy')


In [15]:

# Обучим модель с использованием GridSearchCV
grid_search.fit(X_train, y_train)

# Выведем наилучшие параметры
print("Наилучшие параметры для GridSearchCV:", grid_search.best_params_)


Наилучшие параметры для GridSearchCV: {'n_neighbors': 25}


In [16]:

# Оценим качество модели с наилучшими параметрами на тестовом наборе данных
y_pred_grid = grid_search.predict(X_test)
print("Accuracy на тестовом наборе данных с использованием GridSearchCV:", accuracy_score(y_test, y_pred_grid))
print("Отчет о классификации на тестовом наборе данных с использованием GridSearchCV:")
print(classification_report(y_test, y_pred_grid))

Accuracy на тестовом наборе данных с использованием GridSearchCV: 0.8063261336881974
Отчет о классификации на тестовом наборе данных с использованием GridSearchCV:
              precision    recall  f1-score   support

       <=50K       0.81      0.99      0.89      7479
        >50K       0.82      0.22      0.35      2290

    accuracy                           0.81      9769
   macro avg       0.81      0.60      0.62      9769
weighted avg       0.81      0.81      0.76      9769



In [17]:

# Проведем поиск по сетке с использованием RandomizedSearchCV
randomized_search = RandomizedSearchCV(knn_base, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)

# Обучим модель с использованием RandomizedSearchCV
randomized_search.fit(X_train, y_train)

# Выведем наилучшие параметры
print("\nНаилучшие параметры для RandomizedSearchCV:", randomized_search.best_params_)


Наилучшие параметры для RandomizedSearchCV: {'n_neighbors': 25}


In [18]:
# Оценим качество модели с наилучшими параметрами на тестовом наборе данных
y_pred_random = randomized_search.predict(X_test)
print("Accuracy на тестовом наборе данных с использованием RandomizedSearchCV:", accuracy_score(y_test, y_pred_random))
print("Отчет о классификации на тестовом наборе данных с использованием RandomizedSearchCV:")
print(classification_report(y_test, y_pred_random))

Accuracy на тестовом наборе данных с использованием RandomizedSearchCV: 0.8063261336881974
Отчет о классификации на тестовом наборе данных с использованием RandomizedSearchCV:
              precision    recall  f1-score   support

       <=50K       0.81      0.99      0.89      7479
        >50K       0.82      0.22      0.35      2290

    accuracy                           0.81      9769
   macro avg       0.81      0.60      0.62      9769
weighted avg       0.81      0.81      0.76      9769



cross_val

In [19]:
# Сравним метрики качества исходной и оптимальной моделей с использованием кросс-валидации

from sklearn.model_selection import cross_val_score

cv_scores_base = cross_val_score(knn_base, X, y, cv=5)
cv_scores_grid = cross_val_score(grid_search.best_estimator_, X, y, cv=5)
cv_scores_random = cross_val_score(randomized_search.best_estimator_, X, y, cv=5)

print("\nСредняя точность базовой модели (без настройки параметров):", cv_scores_base.mean())
print("Средняя точность модели с наилучшими параметрами GridSearchCV:", cv_scores_grid.mean())
print("Средняя точность модели с наилучшими параметрами RandomizedSearchCV:", cv_scores_random.mean())


Средняя точность базовой модели (без настройки параметров): 0.7764220340814669
Средняя точность модели с наилучшими параметрами GridSearchCV: 0.8009910023089469
Средняя точность модели с наилучшими параметрами RandomizedSearchCV: 0.8009910023089469


StratifiedKFold


In [20]:
from sklearn.model_selection import StratifiedKFold

# Инициализируем StratifiedKFold с 5 фолдами
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores_skf_base = cross_val_score(knn_base, X, y, cv=skf)
cv_scores_skf_grid = cross_val_score(grid_search.best_estimator_, X, y, cv=skf)
cv_scores_skf_random = cross_val_score(randomized_search.best_estimator_, X, y, cv=skf)

print("\nСредняя точность базовой модели (без настройки параметров) с использованием StratifiedKFold:", cv_scores_skf_base.mean())
print("Средняя точность модели с наилучшими параметрами GridSearchCV с использованием StratifiedKFold:", cv_scores_skf_grid.mean())
print("Средняя точность модели с наилучшими параметрами RandomizedSearchCV с использованием StratifiedKFold:", cv_scores_skf_random.mean())



Средняя точность базовой модели (без настройки параметров) с использованием StratifiedKFold: 0.7783669032287109
Средняя точность модели с наилучшими параметрами GridSearchCV с использованием StratifiedKFold: 0.800847608000336
Средняя точность модели с наилучшими параметрами RandomizedSearchCV с использованием StratifiedKFold: 0.800847608000336
